# The Basics of Cython

#### This notebook is a tutorial on the fundamental basics of Cython.

The most basic nature of Cython can be stated as: ___Cython is Python with C data types.___

There are several features of Cython that we need to be aware of:

1. ___Cython is Python:___ Almost any piece of Python code is also valid Cython code. 

2. The Cython compiler converts Python code into C code which makes equivalent calls ot the Python/C API (*this is partially how __Cython is Python__*).

3. Conversions of Python values and C values occurs automatically wherever possible. In addition to this, ___error checking___ of Python operations and ___reference counting___ are also done automatically.

4. Python's full power of exception handling is available in Cython, i.e., the `try`-except and `try`-finally statements, all in the midst of handling C data.

So, what we can expect is that we will be able to write our usual Python code but this time we can add an extra touch of flair to it with C data types. This will make our code run faster, especially when we are working with large datasets.

Without wasting too much time, let's get cracking!

***

## Our first progam: "Hello World!"


We'll start out by attempting to print `"Hello World!"`. To do this, we need several things:

1. the source file with the extension `.pyx`
2. the setup or compile file - an equivalent to 'makefile'

Our source file will have the following code:

```python
print('Hello World!')
```

while our setup file will have the following:

```python
# Importing
from distutils.core import setup
from Cython.Build import cythonize
# Setup
setup(ext_modules=cythonize('hello_world.pyx'))
```

We'll then use the following command to compile our code:

```bash
python3 setup.py build_ext --place
```

which will result in the creation of the `hello_world.so` file in unix or `hello_world.pyd` in Windows.

In [1]:
# Our 'hello_world.pyx' file - this is the file with the actual source
!touch hello_world.pyx
!echo "print('Hello World!')" >> hello_world.pyx

# Our 'setup.py' file - this is similar to a makefile but it's for Python
!touch setup.py
!echo "from distutils.core import setup" >> setup.py
!echo "from Cython.Build import cythonize" >> setup.py
!echo "setup(ext_modules=cythonize('hello_world.pyx'))" >> setup.py

In [4]:
# Now we can build the Cython file
!python3 setup.py build_ext --inplace
!ls

running build_ext
basics_of_cython.ipynb            hello_world.cpython-36m-darwin.so
build                             hello_world.pyx
hello_world.c                     setup.py


In [3]:
# Importing the 'hello_world.pyx' file - just like a regular Python module
import hello_world

Hello World!


Voila! We have just written and built our first Cython program. However, with this program, we have barely scratched the surface of Cython or from writing any useful code.

***

## `pyximport` : Cython Compilation for Developers

If the module we are building does not require any extra C libraries or a special approach to the setup, we can simply use the `pyximport` module to load `.pyx` files directly from import. This can be done without having to run out `setup.py` file each time we change our code.

We'll write a new 'hello world' program but with a different name to test this out.

In [8]:
# Our 'hello_world.pyx' file - this is the file with the actual source
!touch hello_world_2.pyx
!echo "print('Hello World!')" >> hello_world_2.pyx

In [9]:
# Importing ...
import pyximport

# Initialization ...
pyximport.install()

# Our program
import hello_world_2

Hello World!


And that's all we need to do with `pyximport`. It simplifies the whole process of code compilation but we maintain the same Python code we already know how to write.

***

## Fibonacci Fun

In [15]:
# Our fibonacci file
!touch fibonacci.pyx
!echo  "def fib(n):\n\t'''Print the Fibonacci series up to n.'''\n\ta,b =0, 1\n\twhile b<n:\n\t\tprint(b)\n\t\ta,b=b,a+b" >> fibonacci.pyx

# The setup ...
!touch setup_2.py
!echo "from distutils.core import setup" >> setup_2.py
!echo "from Cython.Build import cythonize" >> setup_2.py
!echo "setup(ext_modules=cythonize('fibonacci.pyx'))" >> setup_2.py

In [17]:
# Building ...
!python3 setup_2.py build_ext --inplace
!ls

running build_ext
basics_of_cython.ipynb            hello_world.cpython-36m-darwin.so
build                             hello_world.pyx
fibonacci.c                       hello_world_2.pyx
fibonacci.cpython-36m-darwin.so   setup.py
fibonacci.pyx                     setup_2.py
hello_world.c


In [38]:
# Importing ...
import fibonacci
fibonacci.fib(50)

1
1
2
3
5
8
13
21
34


## Primes

In [32]:
# Primes ...
!touch primes.pyx
!echo "def primes(int kmax):\n\tcdef int n, k, i\n\tcdef int p[1000]\n\tresult = list()\n\tif (kmax > 1000):\n\t\tkmax = 1000\n\tk, n = 0, 2\n\twhile (k < kmax):\n\t\ti = 0\n\t\twhile (i < k) and (n % p[i] != 0):\n\t\t\ti = i + 1\n\t\tif (i == k):\n\t\t\tp[k] = n\n\t\t\tk = k + 1\n\t\t\tresult.append(n)\n\t\tn = n + 1\n\treturn result" >> primes.pyx

# The setup ...
!touch setup_3.py
!echo "from distutils.core import setup" >> setup_3.py
!echo "from Cython.Build import cythonize" >> setup_3.py
!echo "setup(ext_modules=cythonize('primes.pyx'))" >> setup_3.py

In [34]:
# Building ...
!python3 setup_3.py build_ext --inplace
!ls

running build_ext
basics_of_cython.ipynb            hello_world_2.pyx
build                             primes.c
fibonacci.c                       primes.cpython-36m-darwin.so
fibonacci.cpython-36m-darwin.so   primes.pyx
fibonacci.pyx                     setup.py
hello_world.c                     setup_2.py
hello_world.cpython-36m-darwin.so setup_3.py
hello_world.pyx


In [35]:
# Importing ...
import primes
primes.primes(10)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]